# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 10:20AM,242840,19,8329844,"Emersa Waterbox, LLC",Released
1,Jul 27 2022 10:15AM,242839,12,8329843,Doctors Scientific Organica,Released
2,Jul 27 2022 10:11AM,242838,19,TR-RN-07282022,"GCH Granules USA, Inc.",Released
3,Jul 27 2022 10:02AM,242836,10,0085908254,ISDIN Corporation,Released
4,Jul 27 2022 10:02AM,242836,10,0085908296,ISDIN Corporation,Released
5,Jul 27 2022 10:02AM,242836,10,0085908293,ISDIN Corporation,Released
6,Jul 27 2022 10:02AM,242836,10,0085908292,ISDIN Corporation,Released
7,Jul 27 2022 10:02AM,242836,10,0085908303,ISDIN Corporation,Released
8,Jul 27 2022 10:02AM,242836,10,0085908315,ISDIN Corporation,Released
9,Jul 27 2022 10:02AM,242836,10,0085908325,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,242836,Released,11
28,242837,Released,1
29,242838,Released,1
30,242839,Released,1
31,242840,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242836,NaN,NaN,11.0
242837,NaN,NaN,1.0
242838,NaN,NaN,1.0
242839,NaN,NaN,1.0
242840,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242735,12.0,0.0,3.0
242781,2.0,3.0,9.0
242788,0.0,1.0,0.0
242789,0.0,0.0,1.0
242791,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242735,12,0,3
242781,2,3,9
242788,0,1,0
242789,0,0,1
242791,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242735,12,0,3
1,242781,2,3,9
2,242788,0,1,0
3,242789,0,0,1
4,242791,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242735,12,,3
1,242781,2,3,9
2,242788,,1,
3,242789,,,1
4,242791,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC"
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc."
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation
14,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance
15,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC"
20,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC"
44,Jul 27 2022 9:43AM,242807,15,"Alliance Pharma, Inc."
63,Jul 27 2022 9:22AM,242833,10,"Nextsource Biotechnology, LLC"
65,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC",,,1
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica,,,1
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc.",,,1
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,,,11
4,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,,,1
5,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC",,,5
6,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC",,,24
7,Jul 27 2022 9:43AM,242807,15,"Alliance Pharma, Inc.",,,19
8,Jul 27 2022 9:22AM,242833,10,"Nextsource Biotechnology, LLC",,,2
9,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC",1,,
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica,1,,
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc.",1,,
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,11,,
4,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,1,,
5,Jul 27 2022 9:51AM,242835,15,"Carwin Pharmaceutical Associates, LLC",5,,
6,Jul 27 2022 9:44AM,242808,15,"Brookfield Pharmaceuticals, LLC",24,,
7,Jul 27 2022 9:43AM,242807,15,"Alliance Pharma, Inc.",19,,
8,Jul 27 2022 9:22AM,242833,10,"Nextsource Biotechnology, LLC",2,,
9,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC",1,,
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica,1,,
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc.",1,,
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,11,,
4,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica,1.0,NaN,NaN
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,11.0,NaN,NaN
4,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 10:20AM,242840,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Jul 27 2022 10:15AM,242839,12,Doctors Scientific Organica,1.0,0.0,0.0
2,Jul 27 2022 10:11AM,242838,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Jul 27 2022 10:02AM,242836,10,ISDIN Corporation,11.0,0.0,0.0
4,Jul 27 2022 10:02AM,242837,10,Beach Patient Assistance,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1942530,48.0,14.0,12.0
12,242839,1.0,0.0,0.0
15,728450,48.0,0.0,0.0
19,2428094,24.0,7.0,0.0
20,485600,33.0,7.0,2.0
21,485633,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1942530,48.0,14.0,12.0
1,12,242839,1.0,0.0,0.0
2,15,728450,48.0,0.0,0.0
3,19,2428094,24.0,7.0,0.0
4,20,485600,33.0,7.0,2.0
5,21,485633,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,48.0,14.0,12.0
1,12,1.0,0.0,0.0
2,15,48.0,0.0,0.0
3,19,24.0,7.0,0.0
4,20,33.0,7.0,2.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,12,Released,1.0
2,15,Released,48.0
3,19,Released,24.0
4,20,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,12.0,0.0,0.0,0.0,2.0,0.0
Executing,14.0,0.0,0.0,7.0,7.0,2.0
Released,48.0,1.0,48.0,24.0,33.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,12.0,0.0,0.0,0.0,2.0,0.0
1,Executing,14.0,0.0,0.0,7.0,7.0,2.0
2,Released,48.0,1.0,48.0,24.0,33.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,12.0,0.0,0.0,0.0,2.0,0.0
1,Executing,14.0,0.0,0.0,7.0,7.0,2.0
2,Released,48.0,1.0,48.0,24.0,33.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()